# CHAPTER 1.  Secondary Sort: Introduction

## Spark Solution to Secondary Sort

### Step 3: Connect to the Spark master

- sparkPackages="com.databricks:spark-csv_2.10:1.0.3"  옵션은 csv 파일을 읽어오기 위한   패키지명

In [1]:
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))

sc <- sparkR.init(master="local[*]", sparkPackages="com.databricks:spark-csv_2.10:1.0.3" )
sqlContext <- sparkRSQL.init(sc)


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var

The following objects are masked from ‘package:base’:

    colnames, colnames<-, endsWith, intersect, rank, rbind, sample,
    startsWith, subset, summary, table, transform



Launching java with spark-submit command /usr/local/spark/bin/spark-submit  --packages com.databricks:spark-csv_2.10:1.0.3 sparkr-shell /tmp/RtmpRBcXAZ/backend_port229367227291 


### Step 4: Use the JavaSparkContext to create a JavaRDD

In [18]:
lines <- SparkR:::textFile(sc, 'timeseries.csv')

In [19]:
SparkR:::take(lines, 10)

[[1]]
[1] "x,2,9"

[[2]]
[1] "y,2,5"

[[3]]
[1] "x,1,3"

[[4]]
[1] "y,1,7"

[[5]]
[1] "y,3,1"

[[6]]
[1] "x,3,6"

[[7]]
[1] "z,1,4"

[[8]]
[1] "z,2,8"

[[9]]
[1] "z,3,7"

[[10]]
[1] "z,4,0"

In [20]:
text_df <- read.df(sqlContext=sqlContext, source="text", path="timeseries.csv", delimiter=",", header="true")

In [21]:
head(text_df)

,value
1,"x,2,9"
2,"y,2,5"
3,"x,1,3"
4,"y,1,7"
5,"y,3,1"
6,"x,3,6"


In [22]:
customSchema <- structType(
    structField("name", "string"),    
    structField("time1", "integer"),
    structField("time2", "integer")
)
csv_df <- read.df(sqlContext=sqlContext, source="com.databricks.spark.csv", path="timeseries.csv", header="true", schema = customSchema)

In [23]:
head(csv_df)

,name,time1,time2


### Step 5: Create key-value pairs from the JavaRDD

- R에서는 tuple을 list로 표현함.

In [24]:
make_pairs <- function ( line ) {
    tokens <- strsplit(line, ",")[[1]]
    #time <- tokens[2]
    #value <- tokens[3]
    time <- as.numeric( tokens[2] )
    value <- as.numeric( tokens[3] )
    timevalue <- list( time, value )
    result <- list( tokens[1], timevalue  )
    return ( result )
}

In [25]:
make_pairs( 'z,4,0' )

[[1]]
[1] "z"

[[2]]
[[2]][[1]]
[1] 4

[[2]][[2]]
[1] 0

In [26]:
pairs <- SparkR:::flatMap( lines, make_pairs  )

- 결과가 맞는지 확인해보자.

In [27]:
SparkR:::take(pairs, 35)

[[1]]
[1] "x"

[[2]]
[[2]][[1]]
[1] 2

[[2]][[2]]
[1] 9


[[3]]
[1] "y"

[[4]]
[[4]][[1]]
[1] 2

[[4]][[2]]
[1] 5


[[5]]
[1] "x"

[[6]]
[[6]][[1]]
[1] 1

[[6]][[2]]
[1] 3


[[7]]
[1] "y"

[[8]]
[[8]][[1]]
[1] 1

[[8]][[2]]
[1] 7


[[9]]
[1] "y"

[[10]]
[[10]][[1]]
[1] 3

[[10]][[2]]
[1] 1


[[11]]
[1] "x"

[[12]]
[[12]][[1]]
[1] 3

[[12]][[2]]
[1] 6


[[13]]
[1] "z"

[[14]]
[[14]][[1]]
[1] 1

[[14]][[2]]
[1] 4


[[15]]
[1] "z"

[[16]]
[[16]][[1]]
[1] 2

[[16]][[2]]
[1] 8


[[17]]
[1] "z"

[[18]]
[[18]][[1]]
[1] 3

[[18]][[2]]
[1] 7


[[19]]
[1] "z"

[[20]]
[[20]][[1]]
[1] 4

[[20]][[2]]
[1] 0


[[21]]
[1] "p"

[[22]]
[[22]][[1]]
[1] 2

[[22]][[2]]
[1] 6


[[23]]
[1] "p"

[[24]]
[[24]][[1]]
[1] 4

[[24]][[2]]
[1] 7


[[25]]
[1] "p"

[[26]]
[[26]][[1]]
[1] 1

[[26]][[2]]
[1] 9


[[27]]
[1] "p"

[[28]]
[[28]][[1]]
[1] 6

[[28]][[2]]
[1] 0


[[29]]
[1] "p"

[[30]]
[[30]][[1]]
[1] 7

[[30]][[2]]
[1] 3

### Step 7: Group JavaPairRDD elements by the key (name)

- groupByKey(rdd, numPartitions)
    - rdd : The RDD to group. Should be an RDD where each element is list(K, V) or c(K, V).
    - numPartitions	: Number of partitions to create.

In [28]:
groups <- SparkR:::groupByKey( pairs, 1L )

In [29]:
SparkR:::take(groups, 30)

[[1]]
[[1]][[1]]
[1] 1

[[1]][[2]]
[[1]][[2]][[1]]
[1] 3

[[1]][[2]][[2]]
[1] 7

[[1]][[2]][[3]]
[1] 4

[[1]][[2]][[4]]
[1] 9



[[2]]
[[2]][[1]]
[1] 2

[[2]][[2]]
[[2]][[2]][[1]]
[1] 9

[[2]][[2]][[2]]
[1] 5

[[2]][[2]][[3]]
[1] 8

[[2]][[2]][[4]]
[1] 6



[[3]]
[[3]][[1]]
[1] 3

[[3]][[2]]
[[3]][[2]][[1]]
[1] 1

[[3]][[2]][[2]]
[1] 6

[[3]][[2]][[3]]
[1] 7



[[4]]
[[4]][[1]]
[1] 4

[[4]][[2]]
[[4]][[2]][[1]]
[1] 0

[[4]][[2]][[2]]
[1] 7



[[5]]
[[5]][[1]]
[1] 6

[[5]][[2]]
[[5]][[2]][[1]]
[1] 0



[[6]]
[[6]][[1]]
[1] 7

[[6]][[2]]
[[6]][[2]][[1]]
[1] 3



[[7]]
[[7]][[1]]
[1] "p"

[[7]][[2]]
[[7]][[2]][[1]]
[1] "112"

[[7]][[2]][[2]]
[1] "112"

[[7]][[2]][[3]]
[1] "112"

[[7]][[2]][[4]]
[1] "112"

[[7]][[2]][[5]]
[1] "112"



[[8]]
[[8]][[1]]
[1] "x"

[[8]][[2]]
[[8]][[2]][[1]]
[1] "120"

[[8]][[2]][[2]]
[1] "120"

[[8]][[2]][[3]]
[1] "120"



[[9]]
[[9]][[1]]
[1] "y"

[[9]][[2]]
[[9]][[2]][[1]]
[1] "121"

[[9]][[2]][[2]]
[1] "121"

[[9]][[2]][[3]]
[1] "121"



[[10]]
[[10]][[1]]
[1] "z"

[[10]][[2]]
[[10]][[2]][[1]]
[1] "122"

[[10]][[2]][[2]]
[1] "122"

[[10]][[2]][[3]]
[1] "122"

[[10]][[2]][[4]]
[1] "122"

###  검증하기 

In [30]:
pairs2 <- list( 
    list('p', list(2, 6) ), 
    list('p', list(4, 7) ), 
    list('p', list(1, 9) ), 
    list('p', list(6, 0) ),
    list('p', list(7, 3) )
)
pairs2

[[1]]
[[1]][[1]]
[1] "p"

[[1]][[2]]
[[1]][[2]][[1]]
[1] 2

[[1]][[2]][[2]]
[1] 6



[[2]]
[[2]][[1]]
[1] "p"

[[2]][[2]]
[[2]][[2]][[1]]
[1] 4

[[2]][[2]][[2]]
[1] 7



[[3]]
[[3]][[1]]
[1] "p"

[[3]][[2]]
[[3]][[2]][[1]]
[1] 1

[[3]][[2]][[2]]
[1] 9



[[4]]
[[4]][[1]]
[1] "p"

[[4]][[2]]
[[4]][[2]][[1]]
[1] 6

[[4]][[2]][[2]]
[1] 0



[[5]]
[[5]][[1]]
[1] "p"

[[5]][[2]]
[[5]][[2]][[1]]
[1] 7

[[5]][[2]][[2]]
[1] 3

In [31]:
rdd2 <- SparkR:::parallelize(sc, pairs2)
parts2 <- SparkR:::groupByKey(rdd2, 2L)
grouped2 <- collect(parts2)
grouped2

[[1]]
[[1]][[1]]
[1] "p"

[[1]][[2]]
[[1]][[2]][[1]]
[[1]][[2]][[1]][[1]]
[1] 2

[[1]][[2]][[1]][[2]]
[1] 6


[[1]][[2]][[2]]
[[1]][[2]][[2]][[1]]
[1] 4

[[1]][[2]][[2]][[2]]
[1] 7


[[1]][[2]][[3]]
[[1]][[2]][[3]][[1]]
[1] 1

[[1]][[2]][[3]][[2]]
[1] 9


[[1]][[2]][[4]]
[[1]][[2]][[4]][[1]]
[1] 6

[[1]][[2]][[4]][[2]]
[1] 0


[[1]][[2]][[5]]
[[1]][[2]][[5]][[1]]
[1] 7

[[1]][[2]][[5]][[2]]
[1] 3